In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from numpy.random import RandomState
import operator
from scipy.interpolate import make_interp_spline
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import json

from sklearn.metrics import accuracy_score, f1_score, classification_report
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.nn.functional as F
import pandas as pd
import random


import os
import re
import sys
import numpy


from statistics import mean
import math
from torch.autograd import Variable
%matplotlib inline

import numpy as np
from collections import defaultdict
from tqdm import tqdm_notebook
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler,
                              SequentialSampler)

# Bert

In [ ]:
folder_path = "Tuning/bert"

In [ ]:
results_h = f'drive/MyDrive/iSarcasm/{folder_path}/results/results_h.jsonl'
results_h_2 = f'drive/MyDrive/iSarcasm/{folder_path}/results/results_h_2.jsonl'
results_t = f'drive/MyDrive/iSarcasm/{folder_path}/results/results_t.jsonl'
results_t_2 = f'drive/MyDrive/iSarcasm/{folder_path}/results/results_t_2.jsonl'

In [ ]:
ls

drive/  sample_data/


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
results_db_h = pd.read_json(results_h, lines=True)
print('results_db_h')

results_db_h


In [ ]:
results_db_h

,model_name,loss_fn,batch_size,lr,f1_sarcastic,f1_score_sarc,f1_score_sent,epoch,seed
0,16_FL_1e-05_1_16826_alpha,FL,16,0.00001,0.596835,0.725983,0.748570,1,16826
1,16_FL_1e-05_2_16826_alpha,FL,16,0.00001,0.622723,0.755117,0.760668,2,16826
2,16_FL_1e-05_3_16826_alpha,FL,16,0.00001,0.621818,0.757848,0.758255,3,16826
3,16_FL_2e-05_1_16826_alpha,FL,16,0.00002,0.597523,0.729019,0.744957,1,16826
4,16_FL_2e-05_2_16826_alpha,FL,16,0.00002,0.611702,0.749582,0.748557,2,16826
5,16_FL_2e-05_3_16826_alpha,FL,16,0.00002,0.618677,0.760240,0.743984,3,16826
6,16_FL_3e-05_1_16826_alpha,FL,16,0.00003,0.603476,0.734876,0.743708,1,16826
7,16_FL_3e-05_2_16826_alpha,FL,16,0.00003,0.610659,0.750257,0.749393,2,16826
8,16_FL_3e-05_3_16826_alpha,FL,16,0.00003,0.613746,0.756836,0.745065,3,16826
9,16_FL_4e-05_1_16826_alpha,FL,16,0.00004,0.602201,0.733598,0.736722,1,16826


In [ ]:
results_db_t = pd.read_json(results_t, lines=True)
print('results_db_t')

results_db_t


In [ ]:
results_db_t

,model_name,loss_fn,batch_size,lr,f1_sarcastic,f1_score_sarc,f1_score_sent,epoch,seed
0,16_FL_1e-05_1_16826_alpha,FL,16,0.00001,0.613043,0.686251,0.606049,1,16826
1,16_FL_1e-05_2_16826_alpha,FL,16,0.00001,0.626506,0.714527,0.629098,2,16826
2,16_FL_1e-05_3_16826_alpha,FL,16,0.00001,0.627076,0.721213,0.641469,3,16826
3,16_FL_2e-05_1_16826_alpha,FL,16,0.00002,0.619068,0.687846,0.609753,1,16826
4,16_FL_2e-05_2_16826_alpha,FL,16,0.00002,0.629829,0.719213,0.627325,2,16826
5,16_FL_2e-05_3_16826_alpha,FL,16,0.00002,0.621964,0.723793,0.647284,3,16826
6,16_FL_3e-05_1_16826_alpha,FL,16,0.00003,0.612489,0.682542,0.596779,1,16826
7,16_FL_3e-05_2_16826_alpha,FL,16,0.00003,0.623853,0.712792,0.634528,2,16826
8,16_FL_3e-05_3_16826_alpha,FL,16,0.00003,0.616372,0.721403,0.645536,3,16826
9,16_FL_4e-05_1_16826_alpha,FL,16,0.00004,0.619561,0.689775,0.594881,1,16826


# Mix

In [ ]:
folder_path = "Tuning/mix"

In [ ]:
results_h = f'drive/MyDrive/iSarcasm/{folder_path}/results/results_h.jsonl'
results_h_2 = f'drive/MyDrive/iSarcasm/{folder_path}/results/results_h_2.jsonl'
results_t = f'drive/MyDrive/iSarcasm/{folder_path}/results/results_t.jsonl'
results_t_2 = f'drive/MyDrive/iSarcasm/{folder_path}/results/results_t_2.jsonl'

In [ ]:
ls

drive/  sample_data/


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
results_db_h = pd.read_json(results_h, lines=True)
print('results_db_h')

results_db_h


In [ ]:
results_db_h

,model_name,loss_fn,batch_size,lr,f1_sarcastic,f1_score_sarc,epoch,seed
0,16_FL_1e-05_1_16826_alpha_e_1_1e-05_16,FL,16,0.00001,0.616776,0.747137,1,16826
1,16_FL_1e-05_1_16826_alpha_e_2_1e-05_16,FL,16,0.00001,0.623917,0.755828,1,16826
2,16_FL_3e-05_1_16826_alpha_e_1_3e-05_16,FL,16,0.00003,0.602785,0.737560,1,16826
3,16_FL_2e-05_1_16826_alpha_e_1_2e-05_16,FL,16,0.00002,0.617143,0.746780,1,16826
4,16_FL_3e-05_1_16826_alpha_e_2_3e-05_16,FL,16,0.00003,0.615111,0.751971,1,16826
5,16_FL_2e-05_1_16826_alpha_e_2_2e-05_16,FL,16,0.00002,0.620629,0.754329,1,16826
6,16_FL_4e-05_1_16826_alpha_e_1_4e-05_16,FL,16,0.00004,0.609215,0.745096,1,16826
7,16_FL_4e-05_1_16826_alpha_e_2_4e-05_16,FL,16,0.00004,0.607527,0.747667,1,16826
8,16_FL_5e-05_1_16826_alpha_e_1_5e-05_16,FL,16,0.00005,0.615385,0.750095,1,16826
9,32_FL_1e-05_1_16826_alpha_e_1_1e-05_32,FL,32,0.00001,0.621327,0.751771,1,16826


In [ ]:
results_db_t = pd.read_json(results_t, lines=True)
print('results_db_t')

results_db_t


In [ ]:
results_db_t

,model_name,loss_fn,batch_size,lr,f1_sarcastic,f1_score_sarc,epoch,seed
0,16_FL_1e-05_1_16826_alpha_e_1_1e-05_16,FL,16,0.00001,0.630784,0.714671,1,16826
1,16_FL_1e-05_1_16826_alpha_e_2_1e-05_16,FL,16,0.00001,0.639416,0.725792,1,16826
2,16_FL_1e-05_1_16826_alpha_e_3_1e-05_16,FL,16,0.00001,0.627632,0.724376,1,16826
3,16_FL_3e-05_1_16826_alpha_e_1_3e-05_16,FL,16,0.00003,0.629944,0.713579,1,16826
4,16_FL_2e-05_1_16826_alpha_e_1_2e-05_16,FL,16,0.00002,0.635071,0.718564,1,16826
5,16_FL_3e-05_1_16826_alpha_e_2_3e-05_16,FL,16,0.00003,0.627569,0.721025,1,16826
6,16_FL_2e-05_1_16826_alpha_e_2_2e-05_16,FL,16,0.00002,0.633902,0.725838,1,16826
7,16_FL_4e-05_1_16826_alpha_e_1_4e-05_16,FL,16,0.00004,0.630392,0.719994,1,16826
8,16_FL_4e-05_1_16826_alpha_e_2_4e-05_16,FL,16,0.00004,0.616502,0.717532,1,16826
9,16_FL_5e-05_1_16826_alpha_e_1_5e-05_16,FL,16,0.00005,0.617910,0.712715,1,16826
